In [72]:
# packages
import Utils as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Cleaning of yield-curve 

In [73]:
yld = pd.read_csv("Data/yield-curve-rates-1990-2023.csv")
yld

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,12/29/23,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
1,12/28/23,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98
2,12/27/23,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
3,12/26/23,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
4,12/22/23,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8502,1/8/90,NaN,NaN,7.79,NaN,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
8503,1/5/90,NaN,NaN,7.79,NaN,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
8504,1/4/90,NaN,NaN,7.84,NaN,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
8505,1/3/90,NaN,NaN,7.89,NaN,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [74]:
yld["Date"] = pd.to_datetime(yld["Date"], format="%m/%d/%y")
yld = yld.sort_values(["Date"])
bonds = yld[["Date","10 Yr"]].copy()
bonds["10 Yr"] = bonds["10 Yr"] / 100
bonds

,Date,10 Yr
8506,1990-01-02,0.0794
8505,1990-01-03,0.0799
8504,1990-01-04,0.0798
8503,1990-01-05,0.0799
8502,1990-01-08,0.0802
...,...,...
4,2023-12-22,0.0390
3,2023-12-26,0.0389
2,2023-12-27,0.0379
1,2023-12-28,0.0384


Data is now cleaned to contain desired variables and formatted. Only left to calculate monthly returns

In [75]:
# Keep only first date and EOM for the remainder to calc returns. 
eom = bonds.set_index("Date").resample('ME').last()
eom = eom.reset_index()
bonds = pd.concat([bonds[bonds["Date"] == '1990-01-02'],eom], ignore_index=True)

We then find the Returns. This is done using functionality for present value in the utils script.

In [76]:
# Sanity check of functionality.
u.PV(0.9, 0.9,10) 


1.0

In [77]:
# Do shift return: 
bonds["10 Yr m1"] = bonds["10 Yr"].shift()

In [78]:
# Calculate monthly returns.
Ti = 10
Tim1 = 10 - 1/12 # could be done even more precise. 

bonds["10YrReturns"] = u.get_r(bonds["10 Yr m1"],bonds["10 Yr"], Tim1 , Ti)
bonds["10YrReturns_pct"] = bonds["10YrReturns"].copy() * 100
bonds["10YrReturns_Bps"] = bonds["10YrReturns"].copy() * 10000
# This was to check some sanity sizes.

In [79]:
bonds = bonds[bonds["Date"] != pd.to_datetime("1990-01-02")]
bonds = bonds[["Date", "10 Yr", "10YrReturns"]].set_index("Date")


In [80]:
bonds.to_csv("Data_clean/bond_returns.csv")

In [81]:
bonds

,10 Yr,10YrReturns
Date,,
1990-01-31,0.0843,-0.028999
1990-02-28,0.0851,-0.001975
1990-03-31,0.0865,-0.005841
1990-04-30,0.0904,-0.021675
1990-05-31,0.0860,0.032038
...,...,...
2023-08-31,0.0409,-0.007360
2023-09-30,0.0459,-0.036899
2023-10-31,0.0488,-0.019940
